In [1]:
import numpy as np
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords

In [2]:
df = pd.read_csv('data/train.tsv/train.tsv',sep='\t')
df.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [3]:
df['Phrase'][0]

'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .'

In [4]:
import string 

'''
    单词预处理，将单词全部小写，并且去除标点符号
'''
def preprocessing(phrase):
    stop_words = stopwords.words("english")
    lower = [phras.lower() for phras in phrase]    # 将字母全部小写
    no_punct = [text.translate(str.maketrans('','',string.punctuation)) for text in lower]   # 去掉标点符号
    sp = [text.split() for text in no_punct]
    nos = []
    for sl in sp:
        w_list = [w for w in sl if w not in stop_words]
        nos.append(w_list)
    res = [' '.join(lis) for lis in nos]

    return res

df['Phrase'] = preprocessing(df['Phrase'])
df.head()

find file path   : corpora/stopwords.zip/stopwords/


,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,series escapades demonstrating adage good goos...,1
1,2,1,series escapades demonstrating adage good goose,2
2,3,1,series,2
3,4,1,,2
4,5,1,series,2


In [5]:
df['Phrase'][0]

'series escapades demonstrating adage good goose also good gander occasionally amuses none amounts much story'

In [6]:
phrase = df['Phrase'].values.tolist()
sentiment = df['Sentiment'].values.tolist()

dic = {}
for sentence in phrase:
    for word in sentence.split(' '):
        dic[word] = dic.get(word,0) + 1
dic_sorted_list = sorted(dic,key=lambda x:dic[x],reverse=True)
print('频次最高的50个单词:')
word_fre = [(dic_sorted_list[i],dic[dic_sorted_list[i]]) for i in range(50)]
print(word_fre)

频次最高的50个单词:
[('film', 6689), ('movie', 5905), ('nt', 3970), ('one', 3609), ('like', 3071), ('story', 2520), ('rrb', 2438), ('lrb', 2098), ('good', 2043), ('characters', 1882), ('much', 1862), ('time', 1747), ('comedy', 1721), ('even', 1597), ('little', 1575), ('funny', 1522), ('way', 1511), ('life', 1484), ('make', 1396), ('movies', 1345), ('love', 1296), ('new', 1278), ('enough', 1248), ('work', 1243), ('us', 1234), ('bad', 1211), ('', 1154), ('something', 1152), ('would', 1118), ('never', 1114), ('director', 1099), ('many', 1094), ('people', 1073), ('made', 1060), ('best', 1059), ('two', 1032), ('makes', 1019), ('action', 1005), ('may', 986), ('plot', 979), ('films', 979), ('could', 969), ('character', 968), ('see', 957), ('well', 944), ('world', 917), ('better', 913), ('audience', 912), ('drama', 894), ('look', 888)]


In [7]:
# 查看某一个单词在训练集中各个分类所占的数量

phrase = df['Phrase'].values.tolist()
sentiment = df['Sentiment'].values.tolist()

search_word = 'spirit'

result = [0,0,0,0,0]
for i in range(len(phrase)):
    if search_word in phrase[i].split(' '):
        result[sentiment[i]] += 1


print('search word :    "',search_word,'"')
print('=========================')
print('result      raw    5  class : ')
for i in range(5):
    print(i,'\t',result[i])
print('=========================')
print('result transform to 3 class : ')
result_3 = [result[0]+result[1],result[2],result[3]+result[4]]
for i in range(3):
    print(i,'\t',result_3[i])


search word :    " spirit "
result      raw    5  class : 
0 	 4
1 	 24
2 	 92
3 	 146
4 	 40
result transform to 3 class : 
0 	 28
1 	 92
2 	 186


In [8]:
# 查找所有单词(或者前1000个单词)中，情感分明的单词，即在训练集中negative和positive数量差距明显的单词

# 0 -- negative word        2 -- neutral word        4 -- positive word
def search_dic_word(phrase,sentiment,search_word):
    res = [0,0,0,0,0]
    for i in range(len(phrase)):
        if search_word in phrase[i].split(' '):
            res[sentiment[i]] += 1
        
    result = [res[0]+res[1],res[2],res[3]+res[4]]
    

    if (result[0]+result[2]) > 3*result[1]:
        if result[0] > 10*result[2] :
            #print(word,res,result)
            return 0
        elif result[2] > 10*result[0]:
            #print(word,res,result)
            return 4
    return 2

from tqdm import tqdm

significance_word = {'negative':[],'positive':[]}
phrase = df['Phrase'].values.tolist()
sentiment = df['Sentiment'].values.tolist()

for word in tqdm(dic_sorted_list[0:1000]):
    flag = search_dic_word(phrase,sentiment,word)

    if flag == 0:
        significance_word['negative'].append(word)
    elif flag == 4:
        significance_word['positive'].append(word)

100%|██████████| 1000/1000 [01:07<00:00, 14.91it/s]


In [9]:
print('negative word num: ',len(significance_word['negative']))
print('positive word num: ',len(significance_word['positive']))
print('=======================================================================')
print('negative word list : ')
print(significance_word['negative'])
print('-----------------------------------------')
print('positive word list : ')
print(significance_word['positive'])


print('\n***********************************************************************\n')
print('all significant in following list:')
all_significance_word = significance_word['negative']
all_significance_word.extend(significance_word['positive'])
print(all_significance_word)

negative word num:  19
positive word num:  38
negative word list : 
['worst', 'lack', 'flat', 'contrived', 'tired', 'mess', 'pretentious', 'stupid', 'ugly', 'worse', 'loud', 'unfunny', 'waste', 'tedious', 'shallow', 'depressing', 'poorly', 'pointless', 'empty']
-----------------------------------------
positive word list : 
['moving', 'fascinating', 'enjoyable', 'beautiful', 'powerful', 'charming', 'rich', 'solid', 'rare', 'honest', 'beautifully', 'hilarious', 'touching', 'perfectly', 'remarkable', 'terrific', 'gentle', 'wonderful', 'gorgeous', 'creative', 'nice', 'warm', 'excellent', 'engrossing', 'warmth', 'grace', 'unique', 'impressive', 'inventive', 'colorful', 'sensitive', 'meditation', 'captures', 'sexy', 'witty', 'brings', 'heartfelt', 'playful']

***********************************************************************

all significant in following list:
['worst', 'lack', 'flat', 'contrived', 'tired', 'mess', 'pretentious', 'stupid', 'ugly', 'worse', 'loud', 'unfunny', 'waste', '

In [10]:
len(all_significance_word)

57

In [11]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset,TensorDataset
import numpy as np


# 定义数据读取类
class MyDataset(Dataset):
    """
    步骤一：继承paddle.io.Dataset类
    """
    def __init__(self,df,significant_word,vocab_path='data/glove.6B/glove.6B.300d.txt',word_len=300):
        """
        步骤二：实现构造函数，定义数据读取方式，划分训练和测试数据集
        """
        super(MyDataset, self).__init__()
        
        self.vocab_path = vocab_path
        self.wordlen = word_len
        self.significant_word = significant_word
        _, _, self.word_to_vec_map = self.load_glove_embeddings()


        self.data =np.nan_to_num(np.array([self.sentence_to_avg(text) for text in df['Phrase']]),nan=0)


        self.label = [la for la in df['Sentiment']]


    def __getitem__(self, index):
        """
        步骤三:实现__getitem__方法,定义指定index时如何获取数据,并返回单条数据(训练数据，对应的标签)
        """
        d = torch.tensor(self.data[index],dtype=torch.float32)   
        l = torch.tensor(self.label[index],dtype=torch.long)

        return d,l

    def __len__(self):
        """
        步骤四:实现__len__方法:返回数据集总数目
        """
        return self.data.shape[0]
    

    # 将句子转换为向量
    def sentence_to_avg(self,sentence):
        words = sentence.lower().strip().split()
        
        avg = np.zeros(self.wordlen+len(self.significant_word),)
        
        # 直接返回全零向量
        if sentence == '':
            return avg

        for w in words:
            if w in self.word_to_vec_map.keys():  # 如果不在词表里面，则该向量设置为全零
                    sign_feat = np.zeros(len(self.significant_word),)
                    if w in self.significant_word:
                        sign_feat[self.significant_word.index(w)] = 1
                    # 将显著词做成一个one-hot向量连接到word2vec向量后
                    avg += np.concatenate((self.word_to_vec_map[w],sign_feat))
        
        avg = avg / len(words)
        
        return avg

    # 加载GloVe词嵌入
    def load_glove_embeddings(self):
        with open(self.vocab_path, 'r', encoding='utf-8') as f:
            words = set()
            word_to_vec_map = {}
            
            for line in f:
                line = line.strip().split()
                curr_word = line[0]
                words.add(curr_word)
                word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float32)
            
            i = 1
            words_to_index = {}
            index_to_words = {}
            for w in sorted(words):
                words_to_index[w] = i
                index_to_words[i] = w
                i = i + 1
        return words_to_index, index_to_words, word_to_vec_map

d:\Anaconda\envs\ML_py38_CU\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
from sklearn.model_selection import train_test_split

train_df,dev_df = train_test_split(df,test_size=0.1,random_state=42)

In [13]:
train_dataset = MyDataset(train_df,all_significance_word)
train_loader = DataLoader(train_dataset,32,shuffle=True)

dev_dataset = MyDataset(dev_df,all_significance_word)
dev_loader = DataLoader(dev_dataset,32,shuffle=True)

训练NN模型

In [14]:
from importlib import import_module

model_name = 'NN'

x = import_module('models.' + model_name)
config = x.Config()
print('all class number : ',config.num_classes)

all class number :  5


In [15]:
from train import train

model = x.Model(config).to(config.device)

train(config,model,train_loader,dev_loader,dev_loader)

Epoch [1/50]


100%|██████████| 4390/4390 [03:19<00:00, 21.99it/s]


train loss : 0.9673 ,train acc:0.596 , dev loss : 0.9913,dev acc : 0.586 
saving model ...
Epoch [2/50]


100%|██████████| 4390/4390 [03:19<00:00, 22.00it/s]


train loss : 0.8947 ,train acc:0.630 , dev loss : 0.9437,dev acc : 0.614 
saving model ...
Epoch [3/50]


...

train loss : 0.7949 ,train acc:0.679 , dev loss : 0.8974,dev acc : 0.636 
saving model ...
Epoch [5/50]


100%|██████████| 4390/4390 [03:29<00:00, 20.94it/s]


train loss : 0.7626 ,train acc:0.686 , dev loss : 0.9057,dev acc : 0.637 
Epoch [6/50]


  4%|▍         | 196/4390 [00:09<03:15, 21.45it/s]


KeyboardInterrupt: 

In [16]:
df = pd.read_csv('data/test.tsv/test.tsv',sep='\t')
df.head()

,PhraseId,SentenceId,Phrase
0,156061,8545,An intermittently pleasing but mostly routine ...
1,156062,8545,An intermittently pleasing but mostly routine ...
2,156063,8545,An
3,156064,8545,intermittently pleasing but mostly routine effort
4,156065,8545,intermittently pleasing but mostly routine


In [17]:
from tqdm import tqdm

'''
    使用model模型预测test_loader标签
'''
def predict_test_data(config,model,test_loader):
    pred = []
    for X,y in tqdm(test_loader):

        data = X.to(config.device)
        out = model(data)
        lab = out.argmax(dim=1) #argmax():返回最大数的索引


        pred.extend(lab.detach().cpu().numpy().tolist())
    return pred

In [18]:
df = pd.read_csv('data/test.tsv/test.tsv',sep='\t')
df['Phrase'] = preprocessing(df['Phrase'])

df['Sentiment'] = [2] * df.shape[0]

In [19]:
test_dataset = MyDataset(df,all_significance_word)
test_loader = DataLoader(test_dataset,32,shuffle=False)

In [20]:
model_name = 'output/NN_2023-02-18_16-23-57'
print('load_model name : ',model_name)
net = x.Model(config).to(config.device)
net.load_state_dict(torch.load(model_name))

test_pred = predict_test_data(config,net,test_loader)

load_model name :  output/NN_2023-02-18_16-23-57


100%|██████████| 2072/2072 [00:06<00:00, 298.88it/s]


In [21]:
Submission_path = 'data/sampleSubmission.csv'
submission = pd.read_csv(Submission_path)
submission.Sentiment = test_pred

In [22]:
submission.to_csv('NN_submission.csv',index=False)